[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/crunchdao/quickstarters/blob/structural-break/competitions/structural-break/quickstarters/random-submission/random-submission.ipynb)

![Banner](https://raw.githubusercontent.com/crunchdao/quickstarters/refs/heads/structural-break/competitions/structural-break/assets/banner.webp)

# Setup

Get the setup command: https://hub.crunchdao.io/competitions/structural-break/submit/notebook

![Reveal token](https://raw.githubusercontent.com/crunchdao/competitions/refs/heads/structural-break/documentation/animations/hub-reveal-token.gif)

In [ ]:
# Install the Crunch CLI
%pip install --upgrade crunch-cli

# Setup your local environment
!crunch setup --notebook structural-break hello --token aaaabbbbccccddddeeeeffff

In [1]:
# Staging only, will be removed for production
%env API_BASE_URL=https://api.hub.crunchdao.io/
%env WEB_BASE_URL=https://hub.crunchdao.io/
%env CRUNCH_COMPETITIONS_BRANCH=structural-break

env: API_BASE_URL=https://api.hub.crunchdao.io/
env: WEB_BASE_URL=https://hub.crunchdao.io/
env: CRUNCH_COMPETITIONS_BRANCH=structural-break


# Your model

## Setup

In [2]:
import os
import random
import typing

# Import your dependencies
import joblib
import pandas as pd
import sklearn.metrics

In [ ]:
import crunch

# Load the Crunch Toolings
crunch = crunch.load_notebook()

## Data

The data was downloaded when you setup your local environment and is now available in the `data/` directory.

![Data on Colab](https://raw.githubusercontent.com/crunchdao/competitions/refs/heads/structural-break/documentation/animations/colab-data.gif)

In [ ]:
# Load the data simply
X_train, y_train, X_test = crunch.load_data()

### `X_train`

Index:
- `id`: the ID of the dataset
- `time`: arbitrary amount of time sampled regularely

Columns:
- `value`: the timeseries data
- `period`: if you are in an **initial segment** (0) or an **extension segment** (1)

In [5]:
X_train

value  period
id time                  
0  0    -0.008200       0
   1    -0.002554       0
   2     0.004278       0
   3     0.003938       0
   4    -0.011060       0
...           ...     ...
79 2351 -0.018265       1
   2352 -0.007344       1
   2353 -0.007398       1
   2354 -0.029814       1
   2355  0.028297       1

[155946 rows x 2 columns]

### `y_train`

This is a simple `pandas.Series` that tells if a dataset id has a structural breakpoint or not.

Index:
- `id`: the ID of the dataset

Value:
- `structural_breakpoint`: the value you need to predict

In [6]:
y_train

id
0      True
1     False
2     False
3      True
4     False
      ...  
75    False
76    False
77     True
78    False
79    False
Name: structural_breakpoint, Length: 80, dtype: bool

### `X_test`

This is a **`list` of `pandas.DataFrame`** that have the same format as [`X_train`](#X_train).

It is provided as a list to make sure you are encouraged to read the records **one by one**, __as this will be mandatory in the [`infer()`](#infer) function__.

In [7]:
print("Number of datasets:", len(X_test))

Number of datasets: 5


In [8]:
X_test[0]

value  period
id time                  
80 0     0.011090       0
   1    -0.007110       0
   2    -0.035688       0
   3     0.009437       0
   4     0.002980       0
...           ...     ...
   1541  0.014189       1
   1542  0.002086       1
   1543 -0.008756       1
   1544  0.013921       1
   1545 -0.011898       1

[1546 rows x 2 columns]

## Implementation

### `train()`

In the training function, users build and train the model to make inferences on the test data. <br />
Your model must be stored in the `model_directory_path`.

In [9]:
def train(
    X_train: pd.DataFrame,
    y_train: pd.Series,
    model_directory_path: str,
):
    model = ...

    joblib.dump(model, os.path.join(model_directory_path, 'model.joblib'))

### `infer()`

In the inference function, the trained model is loaded and used to make inferences on a sample of data that matches the characteristics of the training test.

#### Setup

Once your model is loaded, you must do a `yield` to signal it to the runner. <br />
After that you can start reading data from `X_test`.

#### Iteration

The datasets must be read **one by one** and each value must be returned with a `yield <value>`. <br />
If you try to skip this, you will get an error. <br />
All values are then concatenated into a prediction file.

**Warning: The datasets can only be iterated once!**

#### Cleanup

Code can be executed after the `for` loop if you need to persist state or do some cleanup.

In [84]:
def infer(
    X_test: typing.Iterable[pd.DataFrame],
    model_directory_path: str,
):
    model = joblib.load(os.path.join(model_directory_path, 'model.joblib'))

    yield  # mark as ready

    # X_test can only be iterated once.
    # Before getting the next dataset, you must predict the current one.
    for dataset in X_test:
        # prediction = model.predict(dataset)
        prediction = round(random.random(), 2)

        yield prediction  # send the prediction for the current dataset

## Local testing

To make sure your `train()` and `infer()` function are working properly, you can call the `.test()` function that will reproduce the cloud environment locally. <br />
Even if it is not perfect, it should give you a quick idea if your model is working properly.

In [ ]:
crunch.test(
    no_determinism_check=True,
)

## Results

Once the local tester is done, you can preview the result stored in `data/prediction.parquet`.

In [ ]:
prediction = pd.read_parquet("data/prediction.parquet")
prediction

### Local scoring

You can call the function that the system uses to estimate your score locally.

In [ ]:
# Load the targets
target = pd.read_parquet("data/y_test.reduced.parquet")["structural_breakpoint"].astype(float)

# Call the scoring function
sklearn.metrics.roc_auc_score(
    target,
    prediction,
)

# Submitting

To submit your work to the platform, you must

### 1. Download your notebook

![Download Notebook on Colab](https://raw.githubusercontent.com/crunchdao/competitions/refs/heads/structural-break/documentation/animations/colab-download-notebook.gif)


### 2. Upload it to the platform

Submit it on: https://hub.crunchdao.io/competitions/structural-break/submit

![Submit Notebook](https://raw.githubusercontent.com/crunchdao/competitions/refs/heads/structural-break/documentation/animations/hub-submit-notebook.gif)